# Problem 1: Data from yfinance

Importing the [yfinance](https://github.com/ranaroussi/yfinance) package.

In [3]:
import yfinance as yf

Getting each data in variables.

In [4]:
# yf.download('META', 'AAPL', 'AMZN', 'NFLX', 'GOOG', period='5d')
#meta_data = yf.download('META', period='5d')
# apple_data = yf.download('AAPL', period='5d')
# amazon_data = yf.download('AMZN', period='5d')
# netflix = yf.download('NFLX', period='5d')
# google_data = yf.download('GOOG', period='5d')

# Using `download` method to get data for 5 days period
# https://ranaroussi.github.io/yfinance/reference/api/yfinance.download.html#yfinance-download
df = yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG'], period='5d')

/tmp/ipykernel_9104/2334644856.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG'], period='5d')
[                       0%                       ]

[*********************100%***********************]  5 of 5 completed


In [5]:
print(df)

Price            Close                                                   \
Ticker            AAPL        AMZN        GOOG        META         NFLX   
Date                                                                      
2025-10-22  258.450012  217.949997  252.529999  733.409973  1116.369995   
2025-10-23  259.579987  221.089996  253.729996  734.000000  1113.589966   
2025-10-24  262.820007  224.210007  260.510010  738.359985  1094.689941   
2025-10-27  268.809998  226.970001  269.929993  750.820007  1094.560059   
2025-10-28  269.000000  229.250000  268.429993  751.440002  1102.500000   

Price             High                                                   ...  \
Ticker            AAPL        AMZN        GOOG        META         NFLX  ...   
Date                                                                     ...   
2025-10-22  262.850006  220.009995  257.179993  740.599976  1157.599976  ...   
2025-10-23  260.619995  221.300003  255.860001  742.409973  1127.829956  ...   

In [7]:
df.to_csv('../data/data.csv')

OSError: Cannot save file into a non-existent directory: '../data'